<a href="https://colab.research.google.com/github/lauraemmanuella/AprendizadoMaquina/blob/main/8_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#monte o drive caso precise usar/salvar arquivos do seu drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#mude o ponto de execução para outro diretório, caso deseje
%cd /content/drive/MyDrive/Colab Notebooks/AprendizadoMaquina

/content/drive/MyDrive/Colab Notebooks/AprendizadoMaquina


# MultiLayer Perceptron

https://scikit-learn.org/stable/modules/neural_networks_supervised.html

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

https://playground.tensorflow.org/


In [3]:
import pandas as pd

dfIris = pd.read_csv('./bases/iris.csv')

In [4]:
#Vamos fazer a mudança de tipo da coluna alvo usando LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
dfIris['class'] = le.fit_transform(dfIris['class']) #ajusta a função aos dados

In [5]:
#dividindo a base para um teste simples por enquanto
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfIris.drop(columns=['class']), dfIris['class'],test_size=0.33, stratify=dfIris['class'])

In [15]:
mlp = MLPClassifier() #configuração default
mlp.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [20]:
from sklearn import metrics
pred = mlp.predict(X_test)
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      0.88      0.93        16
           2       0.89      1.00      0.94        17

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50



## Escolha dos melhores parâmetros usando GridSearchCV

In [6]:
param_grid = {
    'hidden_layer_sizes': [(100, ), (40,), (20,), (50, 25)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [200, 1000, 5000, 10000]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

grid = GridSearchCV(MLPClassifier(),param_grid, verbose = 1)

grid.fit(X_train,y_train)

In [8]:
grid.best_params_

{'activation': 'identity',
 'hidden_layer_sizes': (100,),
 'learning_rate_init': 0.001,
 'max_iter': 1000,
 'solver': 'adam'}

In [10]:
grid.best_score_

0.99

In [11]:
from sklearn import metrics

pred = grid.predict(X_test)
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.82      0.90        17
           2       0.85      1.00      0.92        17

    accuracy                           0.94        50
   macro avg       0.95      0.94      0.94        50
weighted avg       0.95      0.94      0.94        50



In [12]:
dfGridSearch = pd.DataFrame(grid.cv_results_)

In [13]:
dfGridSearch.loc[dfGridSearch['rank_test_score'] == 1, :]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate_init,param_max_iter,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.334366,0.013363,0.001412,0.000242,identity,"(100,)",0.001,1000,adam,"{'activation': 'identity', 'hidden_layer_sizes...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
4,0.418099,0.003722,0.001456,0.000123,identity,"(100,)",0.001,5000,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
5,0.289986,0.036232,0.001309,0.000086,identity,"(100,)",0.001,5000,adam,"{'activation': 'identity', 'hidden_layer_sizes...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
7,0.301535,0.021064,0.001741,0.000425,identity,"(100,)",0.001,10000,adam,"{'activation': 'identity', 'hidden_layer_sizes...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
8,0.084845,0.003355,0.001931,0.000347,identity,"(100,)",0.01,200,sgd,"{'activation': 'identity', 'hidden_layer_sizes...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0.130263,0.025489,0.001417,0.000134,relu,"(20,)",0.01,10000,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
281,0.078361,0.016061,0.001804,0.000448,relu,"(20,)",0.1,200,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
283,0.085834,0.024629,0.001709,0.000277,relu,"(20,)",0.1,1000,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
285,0.068684,0.006382,0.001301,0.000021,relu,"(20,)",0.1,5000,adam,"{'activation': 'relu', 'hidden_layer_sizes': (...",0.95,1.0,1.0,1.0,1.0,0.99,0.02,1
